In [ ]:
%load_ext autoreload
%autoreload 2

from imports import *
from models import *
from utils import *
from data import *
from configs import CONFIGS, EXP_CODES

In [ ]:
'''
load saved experiment
'''


config_name = 'freq'
config = deepcopy(CONFIGS[config_name])

train_dataset, test_dataset, train_loader, test_loader = get_dataset_dataloader(config)

In [ ]:
for i in range(0, 80, 15):
    plt.figure()
    data = train_dataset[1800][0][i].squeeze()

    plt.plot(data, lw=5.0)#, cmap='gray')

    plt.tight_layout()

    ax = plt.gca()
    # ax.xaxis.set_tick_params(labelbottom=False)
    # ax.yaxis.set_tick_params(labelleft=False)
    # ax.set_xticks([])
    # ax.set_yticks([])

    ax = plt.gca()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Remove the bounding box (spines)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.savefig('./freq{}.png'.format(i // 15))
    

#     x_min, x_max = 0, 100
#     y_min, y_max = data.min(), data.max()

#     # Adjust the x and y axis lines to match the data range
#     ax.annotate('', xy=(x_max, 0), xytext=(0, 0),
#             arrowprops=dict(arrowstyle="->", lw=1.5, color='black', shrinkA=0, shrinkB=0, mutation_scale=15))

#     ax.annotate('', xy=(0, y_max), xytext=(0, 0),
#             arrowprops=dict(arrowstyle="->", lw=1.5, color='black', shrinkA=0, shrinkB=0, mutation_scale=15))

#     # Adjust limits to show only the first quadrant
#     ax.set_xlim(0, x_max)
#     ax.set_ylim(0, y_max)

#     ax.grid(False)

In [ ]:
config_name = 'freq'
config = deepcopy(CONFIGS[config_name])

train_dataset, test_dataset, train_loader, test_loader = get_dataset_dataloader(config)

In [ ]:
# assume `frames_tensor` is tensor of Nx12x12 grayscale images

def display_video(frames_tensor, fps=10):
    num_frames, height, width = frames_tensor.shape

    fig = plt.figure(figsize=(width / 20, height / 20))
    ax = plt.axes()

    # Define the function to update the frame
    def update(frame):
        ax.imshow(frames_tensor[frame], cmap='gray')#, vmin=0, vmax=1.0)
        ax.axis('off')

    # Create the animation
    anim = animation.FuncAnimation(fig, update, frames=num_frames, interval=1000/fps)

    # Display the animation
    plt.close()
    return HTML(anim.to_jshtml())

In [ ]:
display_video(train_dataset[100][0].squeeze(dim=1))

In [ ]:
# norm_diff = []
# norm_diff_normalized = []
# cosine_sim = []
# ssim = []
# norm_vs = []

one_two, three_two, one_three, three_one, two_four, four_two = [], [], [], [], [], []
one_two_vs, three_two_vs, one_three_vs, three_one_vs, two_four_vs, four_two_vs = [], [], [], [], [], []

for data, vs in train_dataset:
    i1, i2, i3 = get_i1_i2_i3(data)

    i1 = i1.squeeze().flatten(start_dim=1, end_dim=-1)
    i2 = i2.squeeze().flatten(start_dim=1, end_dim=-1)
    i3 = i3.squeeze().flatten(start_dim=1, end_dim=-1)

    i2p = i2[:-1, :]
    i4p = i2[1:, :]
    
    
    #norm_diff.append(torch.norm(i1 - i2, dim=-1))
    #norm_diff_normalized.append( torch.norm(i1 - i2, dim=-1) / (torch.norm(i1, dim=-1) + torch.norm(i2, dim=-1)) )
    one_two.append(cosine_dissim(i1, i2))
    one_two_vs.append(vs[::2, :])
    # three_two.append(1 - nn.CosineSimilarity(dim=1)(i3, i2))
    one_three.append(cosine_dissim(i1, i3))
    one_three_vs.append(2*vs[::2, :])
    # three_one.append(1 - nn.CosineSimilarity(dim=1)(i3, i1))
    # two_four.append(1 - nn.CosineSimilarity(dim=1)(i2p, i4p))
    # four_two.append(1 - nn.CosineSimilarity(dim=1)(i4p, i2p))
    # ssim.append(1 - SSIM(data_range=1.0, size_average=False, channel=1, nonnegative_ssim=True)(
        # i1.view(-1, 1, 32, 12), i2.view(-1, 1, 32, 12)
    # ))

    # norm_vs.append(vs[::2, :].norm(p=2, dim=-1))
    #     

# norm_diff = torch.cat(norm_diff).numpy()
# norm_diff_normalized = torch.cat(norm_diff_normalized).numpy()
one_two = torch.cat(one_two).numpy()
one_two_vs = torch.cat(one_two_vs).numpy()
# three_two = torch.cat(three_two).numpy()
one_three = torch.cat(one_three).numpy()
one_three_vs = torch.cat(one_three_vs).numpy()
# three_one = torch.cat(three_one).numpy()
# two_four = torch.cat(two_four).numpy()
# four_two = torch.cat(four_two).numpy()

# ssim = torch.cat(ssim).numpy()

# norm_vs = torch.cat(norm_vs).numpy()

In [ ]:
scatter = one_two_vs[one_two < 1e-4]
# scatter = np.hstack((scatter, np.ones((scatter.shape[0], 1))))

plt.scatter(*scatter.T)

print(scatter.shape[0] / len(one_two))
print(scatter[:, 0].max())

### Walking along the manifold of Gaussian blob images

In [ ]:
model = FC_FC(seed=2).cuda()

model.load_state_dict(torch.load('./models/model_lzhu4svj.pt'))

_ = model.eval()

In [ ]:
train_dataset, test_dataset, train_loader, test_loader = get_dataset_dataloader(CONFIGS['fc_fc_3dim_gauss_loops'])

In [ ]:
steps = 100

with torch.no_grad():
    rand_ind = torch.randint(0, len(train_dataset), size=(1,)).item()
    i1, i2, i3 = get_i1_i2_i3(train_dataset[rand_ind][0])

    i1 = i1.cuda()
    i2 = i2.cuda()

    i1 = convert_range(i1, (i1.min(), i1.max()), (0, 1))
    i2 = convert_range(i2, (i2.min(), i2.max()), (0, 1))

    rand_ind = torch.randint(0, i1.shape[1], size=(1,)).item()
    v, pred_i3 = model(i1[0][rand_ind], i2[0][rand_ind])

    pred_i3s = [convert_range(pred_i3, (pred_i3.min(), pred_i3.max()), (0, 1))]

    for _ in range(steps):
        pred_i3 = model.get_i3(v, pred_i3s[-1])
        pred_i3 = convert_range(pred_i3, (pred_i3.min(), pred_i3.max()), (0, 1))
        pred_i3s.append(pred_i3)

    # for _ in range(steps):
    #     pred_i3 = model.get_i3(-v, pred_i3s[-1])
    #     pred_i3 = convert_range(pred_i3, (pred_i3.min(), pred_i3.max()), (0, 1))
    #     pred_i3s.append(pred_i3)

    pred_i3s = torch.stack(pred_i3s).squeeze().cpu()


def display_video(frames_tensor, fps=10):
    num_frames, height, width = frames_tensor.shape

    fig = plt.figure(figsize=(width / 20, height / 20))
    ax = plt.axes()

    # Define the function to update the frame
    def update(frame):
        ax.imshow(frames_tensor[frame], cmap='gray', vmin=0, vmax=1.0)
        ax.axis('off')

    # Create the animation
    anim = animation.FuncAnimation(fig, update, frames=num_frames, interval=1000/fps)

    # Display the animation
    plt.close()
    return HTML(anim.to_jshtml())


# assume `frames_tensor` is tensor of Nx12x12 grayscale images
display_video(pred_i3s, fps=10)

In [ ]:
with torch.no_grad():
    rand_ind = torch.randint(0, len(train_dataset), size=(1,)).item()
    i1, i2, i3 = get_i1_i2_i3(train_dataset[rand_ind][0])

    i1 = i1.cuda()
    i2 = i2.cuda()

    rand_ind = torch.randint(0, i1.shape[1], size=(1,)).item()
    v, pred_i3 = model(i1[0][rand_ind], i2[0][rand_ind])

    pred_vs = [v]
    i1s = [i2[0][rand_ind]]
    i2s = [pred_i3]

    for _ in range(100):
        v, pred_i3 = model(i1s[-1], i2s[-1])

        i1s.append(i2s[-1])
        i2s.append(pred_i3)
        pred_vs.append(v)


    i2s = torch.stack(i2s).squeeze().cpu()


def display_video(frames_tensor, fps=10):
    num_frames, height, width = frames_tensor.shape

    fig = plt.figure(figsize=(width / 20, height / 20))
    ax = plt.axes()

    # Define the function to update the frame
    def update(frame):
        ax.imshow(frames_tensor[frame], cmap='gray', vmin=0, vmax=1.0)
        ax.axis('off')

    # Create the animation
    anim = animation.FuncAnimation(fig, update, frames=num_frames, interval=1000/fps)

    # Display the animation
    plt.close()
    return HTML(anim.to_jshtml())


# assume `frames_tensor` is tensor of Nx12x12 grayscale images
display_video(pred_i3s, fps=10)

In [ ]:
plt.plot(*torch.stack(pred_vs).squeeze().cpu().numpy().T)